In [ ]:
"""
4. O resultado da campanha anterior tem relevância na campanha atual?

"""

In [33]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [28]:
data = pd.read_csv('data/bank-full.csv',sep = ';')
data = data[['poutcome', 'y']]
data = pd.get_dummies(data, columns=['poutcome'])
boolean_dict = {'yes':1, 'no':0}
data.y = data.y.replace(boolean_dict)
#basicão, getdummies tals

In [3]:
x = data.iloc[:,1:].values
y = data.iloc[:,:1].values
#ok, como temos poucas colunas usar iloc tá blza

In [30]:
results = sm.OLS(y, x).fit()
#como temos uma análise de caráter mais qualitativa, não vamos dividir em train/test  

In [31]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     1621.
Date:                Fri, 12 Apr 2019   Prob (F-statistic):               0.00
Time:                        14:01:41   Log-Likelihood:                -10524.
No. Observations:               45211   AIC:                         2.106e+04
Df Residuals:                   45207   BIC:                         2.109e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1261      0.004     28.905      0.0

In [ ]:
"""
beleza, r² baixo já era esperado, dado que é muito difícil uma variável 
explciar outra de forma tão direta. 

Restposta:
O "P" está bem baixo, o que implica que o resultado passado influencia no atual.

---------------------------------------------------------------------------------
Uma observação pessoal:
Pela tabela, há 4 outcomes para um campanha de mkt,sendo que há 3 com 
sentido físico e um que fica sem entendimento.

Explicando um pouco mais, é lógico pensar que a não ação resultaria em nada.
Assim, o outcome unknow deveria ser contabilizado como output dado que?
Ou melhor, 'unknown' deveria ser utilizado na modelagem ? 
"""

In [ ]:
""""
dia  15/04
Ok, talvez um chi quadrado fosse melhor
""""

In [123]:
data = pd.read_csv('data/bank-full.csv',sep = ';')
data = data[['poutcome', 'y']]
# data = pd.get_dummies(data, columns=['poutcome'])

In [124]:
df = data[data['y']=='yes'].groupby('poutcome').count().reset_index()

In [125]:
f_esp = df.y.sum()/4

In [126]:
df['f_esp'] = f_esp
df['f_f'] =((df['y']- df['f_esp'])**2)/f_esp

In [130]:
df

,poutcome,y,f_esp,f_f
0,failure,618,1322.25,375.094016
1,other,307,1322.25,779.529259
2,success,978,1322.25,89.626064
3,unknown,3386,1322.25,3221.073218


In [132]:
df.f_f.sum()

4465.322556248818

In [113]:
gl = 3
chi2_crit = 7.815

In [133]:
"""
como que 4465.32 >>>>> 7.815
"""

'\ncomo que 4465.32 >>>>> 7.815\n'

In [43]:
# data[(data['previous']>0) & (data['previous']<50)]['previous'].hist(bins = 20)
data['boolprevious'] = data['previous'].apply(lambda x: int(1) if x>0 else 0 ) 

In [29]:
data.poutcome.unique()

array(['unknown', 'failure', 'other', 'success'], dtype=object)

In [47]:
data.sample(2)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,boolprevious
1407,43,blue-collar,divorced,secondary,no,338,yes,no,unknown,8,may,116,1,-1,0,unknown,no,0
29706,33,blue-collar,married,secondary,no,640,yes,no,cellular,3,feb,439,3,-1,0,unknown,no,0


In [39]:
data[['boolprevious','poutcome','y']]
data.groupby(['poutcome','boolprevious']).count()

age    job  marital  education  default  balance  \
poutcome boolprevious                                                       
failure  1              4901   4901     4901       4901     4901     4901   
other    1              1840   1840     1840       1840     1840     1840   
success  1              1511   1511     1511       1511     1511     1511   
unknown  0             36954  36954    36954      36954    36954    36954   
         1                 5      5        5          5        5        5   

                       housing   loan  contact    day  month  duration  \
poutcome boolprevious                                                    
failure  1                4901   4901     4901   4901   4901      4901   
other    1                1840   1840     1840   1840   1840      1840   
success  1                1511   1511     1511   1511   1511      1511   
unknown  0               36954  36954    36954  36954  36954     36954   
         1                   5      5        5      5      5         5   

                       campaign  pdays  previous      y  
poutcome boolprevious                                    
failure  1                 4901   4901      4901   4901  
other    1                 1840   1840      1840   1840  
success  1                 1511   1511      1511   1511  
unknown  0                36954  36954     36954  36954  
         1                    5      5         5      5

In [92]:
#P de comprar
p_buy_yes = len(data[data['y']=='yes'])/len(data)
#P de não comprar
p_buy_no = len(data[data['y']=='no'])/len(data)

In [119]:
#P de impactar
p_impact_yes = len(data[data['boolprevious']==1])/len(data)
#P de não impactar
p_impact_no = len(data[data['boolprevious']==0])/len(data)

In [115]:
#impactou e compro
p_yesbuy_yesimpact = len(data[(data['y']=='yes') & (data['boolprevious']== 1)])/len(data[data['boolprevious']== 1])
#impactou e não comprou
p_yesbuy_notimpact = len(data[(data['y']=='no') & (data['boolprevious']== 0)])/len(data[data['boolprevious']== 0])


In [120]:
p_yesbuy_yesimpact

0.2307133341407291

In [122]:
p_yesbuy_yesimpact/p_impact_yes

1.2632651749832267

In [123]:
p_yesbuy_notimpact/p_impact_yes

4.9740679667153955